# M3 N1 Introduction to bokeh with pandas

In this notebook, we'll introduce the `bokeh` plotting package in `python`, wrangle some cloud-hosted data with `pandas`, and introduce some of the methods you'll use in Module 3's homework.

You'll need to install a few new packages for this assignment, so be sure to install them in the class conda environment.

One is `pandas-bokeh`, which can be installed with

```bash
pip install pandas-bokeh
```

All right, for this notebook, and for your assignment, we'll be using the NOAA Global Historical Climatology Network Daily (GHCN-D) database, which is the official record for daily weather observations.  The dataset is produced by the National Centers for Environmental Information https://doi.org/10.7289/V5D21VHZ.  As part of the NOAA Big Data project, this dataset is hosted in the cloud by several popular cloud data services. For this assignment, we'll use the service on Amazon Web Services, who hosts the dataset in an S3 bucket in a few formats. The link to the S3 page for GHCN-D is https://registry.opendata.aws/noaa-ghcn/.

The data is produced in its original format in fixed-width format text files.  We'll use `pandas` to load some metadata - the station inventory file.

In [ ]:
stn_ids = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt', header=None, infer_nrows=1000)
stn_ids.columns = ['ID','LAT','LON','ELEV','UKN','NAME','GSN','WBAN']
stn_ids

The next file we'll open is the station inventory file.  It describes for each site and each variable, the years that each variable is available.

In [ ]:
periods = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-inventory.txt', header=None, infer_nrows=1000)
periods.columns = ['ID','LAT','LON','ELEM','TiMIN','TiMAX']
periods

Let's use `pandas` to merge these two tables into one based on the ID.

In [ ]:
merged_stns = pd.merge(stn_ids,periods,how='left',left_on='ID',right_on='ID')
merged_stns

Let's filter the merged table just for the `TMAX` element, and those stations still reporting in 2022.  Notice how the number of stations drops - most of the stations in the dataset measure rainfall & many stations have stopped reporting.

In [ ]:
#subset for T
merged_stns = merged_stns[(merged_stns['ELEM'] == 'TMAX') & (merged_stns['TiMAX'] == 2022)]
merged_stns


Let's visualize where these currently reporting stations are using a map, depending on when they started taking measurements.  We'll sort them so the oldest ones are last, so they plot on the top of the map.

In [ ]:
merged_stns_sorted = merged_stns.sort_values('TiMIN', ascending=False)

In [ ]:
merged_stns_sorted

Let's do a scatter plot on a `cartopy` map.

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs

fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# make the map global rather than have it zoom in to
# the extents of any plotted data
ax.set_global()
ax.coastlines()

plt.scatter(merged_stns_sorted['LON_x'],merged_stns_sorted['LAT_x'],0.25,c=merged_stns_sorted['TiMIN'], 
            transform=ccrs.PlateCarree(), cmap='magma')
plt.colorbar(shrink = 0.5, label='year')
plt.title('Year of first TMAX measurement in GHCN daily data with stations still reporting in 2022');

Let's do a query to find stations that are reporting in 2022 and contain the string 'CHAMPAIGN' the station name field.

In [ ]:
merged_stns[merged_stns['NAME'].str.contains('CHAMPAIGN', regex=False)]

Let's load the data! The data is avaiable in AWS in a couple of formats.  It has a mirror of the raw data format in column separated values (csv), and pandas can read it without a problem.  I've added a couple of flags to avoid errors, but this should work to open the data in this format.

Again, to learn more, visit the AWS site for GHCN-D (https://registry.opendata.aws/noaa-ghcn/)

Let's load 

In [ ]:
df = pd.read_csv(
...      "s3://noaa-ghcn-pds/csv/by_station/USC00118740.csv",
...      storage_options={"anon": True},  # passed to `s3fs.S3FileSystem`
         dtype={'Q_FLAG': 'object', 'M_FLAG': 'object'},
         parse_dates=['DATE']
... ).set_index('DATE')

Looking at the structure of the data, each "element" takes up one line, there are QC flags for each variables, and other informtion.

In [ ]:
df

We can also load the data in a parquet format, which is a cloud-optimized database format.  It also works well with `dask`.  We won't use `dask` for this exercise because a single site is a small file.  However, if you were interested in looking at larger portions of the dataset, `dask` could help.

In [ ]:
from datetime import datetime

df = pd.read_parquet(
    "s3://noaa-ghcn-pds/parquet/by_station/STATION=USC00118740/",
    storage_options={"anon": True},  # passed to `s3fs.S3FileSystem`
)

#make date the index
df['DATE'] = df['DATE'].apply(lambda x: datetime.strptime(x, '%Y%m%d'))
df = df.set_index('DATE').sort_index()  #we need to sort by time because the files are sorted to be arbitrary
df


Let's create separate dataframes for elements of interest: Maximum and Minimum Temperature and Precipitaion.

In [ ]:
df_tmax = df.loc[df['ELEMENT'] == 'TMAX']
df_tmin = df.loc[df['ELEMENT'] == 'TMIN']
df_prcp = df.loc[df['ELEMENT'] == 'PRCP']

In [ ]:
df_tmax

Note that the values are a not a magnitude you might expect - they are scaled integers.  To get values in degrees C, divide the values by 10.

## Bokeh and pandas plotting

So, let's visualize this data using `bokeh`.  `bokeh` is an alternative to `matplotlib` and it produces interactive high quality plots that can be visualized in the notebook or on a website.  Check out the `bokeh` website to see what it can do: (https://bokeh.org).

First, we'll use the pandas_bokeh library to plot directly from the pandas dataframe.  We'll also have the plots shown in the notebook here, but you should try turning that off and seeing what happens.  If successful, you should see a little badge below the next cell.

In [ ]:
import pandas as pd
import pandas_bokeh

# see what happens when you comment this out and try to make a bokeh plot!
pandas_bokeh.output_notebook()

Ok, let's plot the time series of maximum temperature for the station we loaded above.  We'll add a fancy rangetool to show the data.

In [ ]:
(df_tmax['DATA_VALUE']/10.).plot_bokeh(rangetool=True);

Let's create an annual mean minimum temperature time series using `pandas` groupby.

In [ ]:
ser=df_tmin[~((df_tmin.index.month==2)&(df_tmin.index.day==29))]

bk_bar1 = (ser['DATA_VALUE']/10.).groupby(ser.index.day_of_year).mean().plot_bokeh().line()


It's easy to create basing plots using `pandas_bokeh`, but you can also using the `bokeh` interface to make plots similar to a matplotlib-type plot.  Here, let's show the maximum temperatures for 2022.

In [ ]:
# bokeh basics
from turtle import color
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Time series of maximum temperatures - 2022',
          x_axis_label = 'Time', 
           y_axis_label = 'Temperature (°C)',
           x_axis_type='datetime')

df_2022 = df_tmax[df_tmax.index >= '2022-01-01']
p.line(df_2022.index, df_2022['DATA_VALUE']/10., legend_label="TMAX", line_width=2)
df_2_2022 = df_tmin[df_tmin.index >= '2022-01-01']
p.line(df_2_2022.index, df_2_2022['DATA_VALUE']/10., legend_label="TMIN", line_width=2, color='orange')

# Show the plot
show(p)

Let's make a histogram of low temperatures

In [ ]:
import numpy as np
arr_hist, edges = np.histogram(df_tmin['DATA_VALUE'][(df_tmin.index >= '1991-01-01') & (df_tmin.index < '2020-01-01')]/10., 
                               bins = np.arange(-40,47,2.5), density=True)
# Put the information in a dataframe
temps = pd.DataFrame({'hist': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})

arr_hist, edges = np.histogram(df_tmin['DATA_VALUE'][(df_tmin.index >= '1901-01-01') & (df_tmin.index < '1930-01-01')]/10., 
                               bins = np.arange(-40,47,2.5), density=True)
# Put the information in a dataframe
temps2 = pd.DataFrame({'hist': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})

In [ ]:
# bokeh basics
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Histogram of Low Temperatures at CHAMPAIGN 3S',
          x_axis_label = 'Temperature (°C)', 
           y_axis_label = 'Fraction of Days')

# Add a quad glyph
p.quad(bottom=0, top=temps['hist'], 
       left=temps['left'], right=temps['right'], 
       fill_color='red', line_color='black', alpha=0.5,legend_label="1991-2020")

p.quad(bottom=0, top=temps2['hist'], 
       left=temps2['left'], right=temps2['right'], 
       fill_color='blue', line_color='black', alpha=0.5, legend_label="1901-1930")

# Show the plot
show(p)